In [11]:
import pandas as pd
import json
import os
import glob

In [12]:
def read_file(file_name:str) -> dict:
    with open(file_name, 'r') as handle:
      content = json.load(handle)
    return content

In [13]:
files_dir = r'C:\Users\P70088982\Documents\edgar-crawler\datasets\extracted_filings'
files = glob.glob(files_dir+r'\*')


Ignore previously processed filings

In [61]:
# ai_mentions = glob.glob(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions\*")
# processed_mentions = pd.DataFrame(files, columns=['file_name'])
# ai_mentions = glob.glob(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions\*")

# def mention_exists(file_name:str) -> bool:
#     """
#     Check if a file has been processed
#     :param file_name: str
#     :return: bool
#     """
#     mention_name = "C:\\Users\\P70088982\\Documents\\edgar-crawler\\datasets\\ai_mentions\\"+file_name.split('\\')[-1].replace('.json', '_matches.json')
#     if mention_name in ai_mentions:
#         return mention_name
#     else:
#         return None
# processed_mentions['mentions'] = processed_mentions['file_name'].apply(lambda x: mention_exists(x))
# # Set processing time to yesterday
# processed_mentions['processing_date'] = pd.to_datetime('now', utc=True).date() - pd.DateOffset(days=1)
# processed_mentions.to_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv")

In [71]:
import re
original_search_terms = ["AI", "Artificial Intelligence","A.I", "Machine Learning","ML","Deep Learning","DL","Neural Network","NLP",
    "Natural Language Processing", "Computer Vision", "Robotics",
    "Computing","algorithm", "Chatbot","Recommendation System", "Recommender System", "Image Recognition", "Speech Recognition", "Voice Assistant",
    "Artificial General Intelligence","AGI", "artificial", "comput\w*", "generative", "agent", "deepfake", "\w*learning\w*", "autonom\w*"]

def extract_keywords(file:dict, sections:list=["1", "1A", "3", "7"], search_terms:list=[]):
  search_terms = ["Artificial Intelligence","A\.I", "Machine Learning","Deep Learning","NLP",
    "Natural Language Processing", "Computer Vision", "Chatbot","Recommendation System", "Recommender System", "Image Recognition", "Speech Recognition", "Voice Assistant",
    "Artificial General Intelligence","AGI", "generative", "deepfake"]
  search_pattern = re.compile(r'\b(' + '|'.join(search_terms) + r')\b', re.IGNORECASE)
  document_results = {}
  document_results['company'] = file['company']
  document_results['cik'] = file['cik']
  document_results['filing_date'] = file['filing_date']
  document_results['period_of_report'] = file['period_of_report']
  document_results['filename'] = file['filename']

  filled=False

  for section in sections:
      if f'item_{section}' in file:
            content = file[f'item_{section}']
            matches = list(search_pattern.finditer(content))
            for match in matches:
                keyword = match.group(0)

                # Extract the surrounding sentence
                start_index = content.rfind('.', 0, match.start()) + 1
                end_index = content.find('.', match.end())
                if end_index == -1:  # If no period found, go to the end of the string
                    end_index = len(content)
                sentence = content[start_index:end_index].strip()

                # Extract the paragraph containing the keyword (starting and ending with a newline)
                paragraph_start = content.rfind("\n", 0, match.start()) + 1  # Start of paragraph
                paragraph_end = content.find("\n", match.end())  # End of paragraph
                if paragraph_end == -1:  # If no newline is found after the match, go to the end of the string
                    paragraph_end = len(content)
                paragraph = content[paragraph_start:paragraph_end].strip()

                # Append match details
                if f'{section}_matches' not in document_results:
                    document_results[f'{section}_matches'] = []
                    filled=True
                document_results[f'{section}_matches'].append({
                    "keyword": keyword,
                    "sentence": sentence,
                    "paragraph": paragraph, 
                    "match_id": f'{file["cik"]}_{section}_{keyword}_{start_index}_{end_index}'})
  if not filled:
    document_results = None
  return document_results            

## Matches extraction and saving to files 
Took ~35m for 5 years of data, may take less if files not saved considering everything fit in memory.

In [66]:
files_dir = r'C:\Users\P70088982\Documents\edgar-crawler\datasets\extracted_filings'
files = glob.glob(files_dir+r'\*')

In [97]:
processed_mentions = pd.read_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv").drop(columns=['Unnamed: 0']).set_index('file_name')
existing = processed_mentions.index.tolist()
to_process = [file for file in files if file not in existing]

In [ ]:
json_files = {}
# processed_mentions = pd.read_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv").set_index('file_name')
# to_process = [file for file in files if file not in processed_mentions['file_name'].values]
for i, file in enumerate(to_process):
    example_file = read_file(file)
    keywords = extract_keywords(example_file)
    processed_mentions.loc[file] = pd.Series([None, pd.to_datetime('now', utc=True).date()], index=processed_mentions.columns)
    if keywords:
        json_files[file] = keywords
        # Save the results to a JSON file
        mentions_file = file.replace('.json', '_matches.json').replace('extracted_filings', 'ai_mentions')


        with open(mentions_file, 'w') as f:
            json.dump(keywords, f, indent=4)
        processed_mentions.loc[file, 'mentions'] = mentions_file

    if i % 100 == 0:
        print(f"{i} files processed, {len(json_files)} files with keywords found, {len(to_process)-i} files remaining")
        # Take out of loop if you don't want to save every 100 files
        processed_mentions.to_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv")


0 files processed, 0 files with keywords found, 1406 files remaining
100 files processed, 30 files with keywords found, 1306 files remaining
200 files processed, 57 files with keywords found, 1206 files remaining
300 files processed, 88 files with keywords found, 1106 files remaining
400 files processed, 108 files with keywords found, 1006 files remaining
500 files processed, 128 files with keywords found, 906 files remaining
600 files processed, 156 files with keywords found, 806 files remaining
700 files processed, 184 files with keywords found, 706 files remaining
800 files processed, 224 files with keywords found, 606 files remaining
900 files processed, 253 files with keywords found, 506 files remaining
1000 files processed, 270 files with keywords found, 406 files remaining
1100 files processed, 288 files with keywords found, 306 files remaining
1200 files processed, 303 files with keywords found, 206 files remaining
1300 files processed, 328 files with keywords found, 106 files 

## Dataframe generation

In [104]:
import pandas as pd
import json
import glob 

In [105]:
json_files = {}
files_paths = glob.glob(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions\*")
for i, file in enumerate(files_paths):
    with open(file, 'r') as handle:
        content = json.load(handle)
    json_files[file] = content
    if i % 100 == 0:
        print(f"{i} files loaded, {len(files_paths)-i} files remaining")

0 files loaded, 9092 files remaining
100 files loaded, 8992 files remaining
200 files loaded, 8892 files remaining
300 files loaded, 8792 files remaining
400 files loaded, 8692 files remaining
500 files loaded, 8592 files remaining
600 files loaded, 8492 files remaining
700 files loaded, 8392 files remaining
800 files loaded, 8292 files remaining
900 files loaded, 8192 files remaining
1000 files loaded, 8092 files remaining
1100 files loaded, 7992 files remaining
1200 files loaded, 7892 files remaining
1300 files loaded, 7792 files remaining
1400 files loaded, 7692 files remaining
1500 files loaded, 7592 files remaining
1600 files loaded, 7492 files remaining
1700 files loaded, 7392 files remaining
1800 files loaded, 7292 files remaining
1900 files loaded, 7192 files remaining
2000 files loaded, 7092 files remaining
2100 files loaded, 6992 files remaining
2200 files loaded, 6892 files remaining
2300 files loaded, 6792 files remaining
2400 files loaded, 6692 files remaining
2500 files l

In [106]:
df_rows = []
for file in json_files:
    match_fields = [match for match in json_files[file].keys() if match.endswith('_matches')]
    non_match_fields = [match for match in json_files[file].keys() if not match.endswith('_matches')]
    for match_field in match_fields:
        for match in json_files[file][match_field]:
            row = {}
            row.update({field: json_files[file][field] for field in non_match_fields})
            row['keyword'] = match['keyword']
            row['sentence'] = match['sentence']
            row['match_id'] = match['match_id']
            row['match_field'] = match_field
            df_rows.append(row)


In [107]:
matches_df = pd.DataFrame(df_rows)

In [108]:
matches_df['filing_year'] = pd.to_datetime(matches_df['filing_date']).dt.year 
matches_df['reporting_year'] = pd.to_datetime(matches_df['period_of_report']).dt.year

In [109]:
matches_df.to_csv(r'C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions.csv', index=False)
matches_df.to_excel(r'C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions.xlsx', index=False)

Scirpting

In [ ]:

import pandas as pd
import json
import os
import glob
def read_file(file_name:str) -> dict:
    with open(file_name, 'r') as handle:
      content = json.load(handle)
    return content
files_dir = r'C:\Users\P70088982\Documents\edgar-crawler\datasets\extracted_filings'
files = glob.glob(files_dir+r'\*')

# Ignore previously processed filings
# ai_mentions = glob.glob(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions\*")
# processed_mentions = pd.DataFrame(files, columns=['file_name'])
# ai_mentions = glob.glob(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions\*")

# def mention_exists(file_name:str) -> bool:
#     """
#     Check if a file has been processed
#     :param file_name: str
#     :return: bool
#     """
#     mention_name = "C:\\Users\\P70088982\\Documents\\edgar-crawler\\datasets\\ai_mentions\\"+file_name.split('\\')[-1].replace('.json', '_matches.json')
#     if mention_name in ai_mentions:
#         return mention_name
#     else:
#         return None
# processed_mentions['mentions'] = processed_mentions['file_name'].apply(lambda x: mention_exists(x))
# # Set processing time to yesterday
# processed_mentions['processing_date'] = pd.to_datetime('now', utc=True).date() - pd.DateOffset(days=1)
# processed_mentions.to_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv")
import re
original_search_terms = ["AI", "Artificial Intelligence","A.I", "Machine Learning","ML","Deep Learning","DL","Neural Network","NLP",
    "Natural Language Processing", "Computer Vision", "Robotics",
    "Computing","algorithm", "Chatbot","Recommendation System", "Recommender System", "Image Recognition", "Speech Recognition", "Voice Assistant",
    "Artificial General Intelligence","AGI", "artificial", "comput\w*", "generative", "agent", "deepfake", "\w*learning\w*", "autonom\w*"]

def extract_keywords(file:dict, sections:list=["1", "1A", "3", "7"], search_terms:list=[]):
  search_terms = ["Artificial Intelligence","A\.I", "Machine Learning","Deep Learning","NLP",
    "Natural Language Processing", "Computer Vision", "Chatbot","Recommendation System", "Recommender System", "Image Recognition", "Speech Recognition", "Voice Assistant",
    "Artificial General Intelligence","AGI", "generative", "deepfake"]
  search_pattern = re.compile(r'\b(' + '|'.join(search_terms) + r')\b', re.IGNORECASE)
  document_results = {}
  document_results['company'] = file['company']
  document_results['cik'] = file['cik']
  document_results['filing_date'] = file['filing_date']
  document_results['period_of_report'] = file['period_of_report']
  document_results['filename'] = file['filename']

  filled=False

  for section in sections:
      if f'item_{section}' in file:
            content = file[f'item_{section}']
            matches = list(search_pattern.finditer(content))
            for match in matches:
                keyword = match.group(0)

                # Extract the surrounding sentence
                start_index = content.rfind('.', 0, match.start()) + 1
                end_index = content.find('.', match.end())
                if end_index == -1:  # If no period found, go to the end of the string
                    end_index = len(content)
                sentence = content[start_index:end_index].strip()

                # Extract the paragraph containing the keyword (starting and ending with a newline)
                paragraph_start = content.rfind("\n", 0, match.start()) + 1  # Start of paragraph
                paragraph_end = content.find("\n", match.end())  # End of paragraph
                if paragraph_end == -1:  # If no newline is found after the match, go to the end of the string
                    paragraph_end = len(content)
                paragraph = content[paragraph_start:paragraph_end].strip()

                # Append match details
                if f'{section}_matches' not in document_results:
                    document_results[f'{section}_matches'] = []
                    filled=True
                document_results[f'{section}_matches'].append({
                    "keyword": keyword,
                    "sentence": sentence,
                    "paragraph": paragraph, 
                    "match_id": f'{file["cik"]}_{section}_{keyword}_{start_index}_{end_index}'})
  if not filled:
    document_results = None
  return document_results            
## Matches extraction and saving to files 
# Took ~35m for 5 years of data, may take less if files not saved considering everything fit in memory.
files_dir = r'C:\Users\P70088982\Documents\edgar-crawler\datasets\extracted_filings'
files = glob.glob(files_dir+r'\*')
processed_mentions = pd.read_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv").drop(columns=['Unnamed: 0']).set_index('file_name')
existing = processed_mentions.index.tolist()
to_process = [file for file in files if file not in existing]
json_files = {}
# processed_mentions = pd.read_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv").set_index('file_name')
# to_process = [file for file in files if file not in processed_mentions['file_name'].values]
for i, file in enumerate(to_process):
    example_file = read_file(file)
    keywords = extract_keywords(example_file)
    processed_mentions.loc[file] = pd.Series([None, pd.to_datetime('now', utc=True).date()], index=processed_mentions.columns)
    if keywords:
        json_files[file] = keywords
        # Save the results to a JSON file
        mentions_file = file.replace('.json', '_matches.json').replace('extracted_filings', 'ai_mentions')


        with open(mentions_file, 'w') as f:
            json.dump(keywords, f, indent=4)
        processed_mentions.loc[file, 'mentions'] = mentions_file

    if i % 100 == 0:
        print(f"{i} files processed, {len(json_files)} files with keywords found, {len(to_process)-i} files remaining")
        # Take out of loop if you don't want to save every 100 files
        processed_mentions.to_csv(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\processed_mentions.csv")

## Dataframe generation
import pandas as pd
import json
import glob 

json_files = {}
files_paths = glob.glob(r"C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions\*")
for i, file in enumerate(files_paths):
    with open(file, 'r') as handle:
        content = json.load(handle)
    json_files[file] = content
    if i % 100 == 0:
        print(f"{i} files loaded, {len(files_paths)-i} files remaining")
df_rows = []
for file in json_files:
    match_fields = [match for match in json_files[file].keys() if match.endswith('_matches')]
    non_match_fields = [match for match in json_files[file].keys() if not match.endswith('_matches')]
    for match_field in match_fields:
        for match in json_files[file][match_field]:
            row = {}
            row.update({field: json_files[file][field] for field in non_match_fields})
            row['keyword'] = match['keyword']
            row['sentence'] = match['sentence']
            row['match_id'] = match['match_id']
            row['match_field'] = match_field
            df_rows.append(row)

matches_df = pd.DataFrame(df_rows)
matches_df['filing_year'] = pd.to_datetime(matches_df['filing_date']).dt.year 
matches_df['reporting_year'] = pd.to_datetime(matches_df['period_of_report']).dt.year
matches_df.to_csv(r'C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions.csv', index=False)
matches_df.to_excel(r'C:\Users\P70088982\Documents\edgar-crawler\datasets\ai_mentions.xlsx', index=False)